# Using LAMMPS with iPython and Jupyter

LAMMPS can be run interactively using iPython easily. This tutorial shows how to set this up.

## Installation

1. Download the latest version of LAMMPS into a folder (we will calls this `$LAMMPS_DIR` from now on)
2. Compile LAMMPS as a shared library and enable exceptions and PNG support
   ```bash
   cd $LAMMPS_DIR/src
   make yes-molecule
   make mpi mode=shlib LMP_INC="-DLAMMPS_PNG -DLAMMPS_EXCEPTIONS" JPG_LIB="-lpng"
   ```

3. Create a python virtualenv
   ```bash
   virtualenv testing
   source testing/bin/activate
   ```

4. Inside the virtualenv install the lammps package
   ```
   (testing) cd $LAMMPS_DIR/python
   (testing) python install.py
   (testing) cd   # move to your working directory
   ```

5. Install jupyter and ipython in the virtualenv
   ```bash
   (testing) pip install ipython jupyter
   ```

6. Run jupyter notebook
   ```bash
   (testing) jupyter notebook
   ```

## Example

In [ ]:
from lammps import IPyLammps

In [ ]:
L = IPyLammps()

In [ ]:
# 2d circle of particles inside a box with LJ walls
import math

b = 0
x = 50
y = 20
d = 20

# careful not to slam into wall too hard

v = 0.3
w = 0.08
                
L.units("lj")
L.dimension(2)
L.atom_style("bond")
L.boundary("f f p")

L.lattice("hex", 0.85)
L.region("box", "block", 0, x, 0, y, -0.5, 0.5)
L.create_box(1, "box", "bond/types", 1, "extra/bond/per/atom", 6)
L.region("circle", "sphere", d/2.0+1.0, d/2.0/math.sqrt(3.0)+1, 0.0, d/2.0)
L.create_atoms(1, "region", "circle")
L.mass(1, 1.0)

L.velocity("all create 0.5 87287 loop geom")
L.velocity("all set", v, w, 0, "sum yes")

L.pair_style("lj/cut", 2.5)
L.pair_coeff(1, 1, 10.0, 1.0, 2.5)

L.bond_style("harmonic")
L.bond_coeff(1, 10.0, 1.2)

L.create_bonds("many", "all", "all", 1, 1.0, 1.5)

L.neighbor(0.3, "bin")
L.neigh_modify("delay", 0, "every", 1, "check yes")

L.fix(1, "all", "nve")

L.fix(2, "all wall/lj93 xlo 0.0 1 1 2.5 xhi", x, "1 1 2.5")
L.fix(3, "all wall/lj93 ylo 0.0 1 1 2.5 yhi", y, "1 1 2.5")

In [ ]:
L.image(zoom=1.8)

In [ ]:
L.thermo_style("custom step temp epair press")
L.thermo(100)
output = L.run(40000)
L.image(zoom=1.8)

## Queries about LAMMPS simulation

In [ ]:
L.system

In [ ]:
L.system.natoms

In [ ]:
L.system.nbonds

In [ ]:
L.system.nbondtypes

In [ ]:
L.communication

In [ ]:
L.fixes

In [ ]:
L.computes

In [ ]:
L.dumps

In [ ]:
L.groups

## Working with LAMMPS Variables

In [ ]:
L.variable("a index 2")

In [ ]:
L.variables

In [ ]:
L.variable("t equal temp")

In [ ]:
L.variables

In [ ]:
import sys

if sys.version_info < (3, 0):
    # In Python 2 'print' is a restricted keyword, which is why you have to use the lmp_print function instead.
    x = float(L.lmp_print('"${a}"'))
else:
    # In Python 3 the print function can be redefined.
    # x = float(L.print('"${a}"')")
    
    # To avoid a syntax error in Python 2 executions of this notebook, this line is packed into an eval statement
    x = float(eval("L.print('\"${a}\"')"))
x

In [ ]:
L.variables['t'].value

In [ ]:
L.eval("v_t/2.0")

In [ ]:
L.variable("b index a b c")

In [ ]:
L.variables['b'].value

In [ ]:
L.eval("v_b")

In [ ]:
L.variables['b'].definition

In [ ]:
L.variable("i loop 10")

In [ ]:
L.variables['i'].value

In [ ]:
L.next("i")
L.variables['i'].value

In [ ]:
L.eval("ke")

## Accessing Atom data

In [ ]:
L.atoms[0]

In [ ]:
[x for x in dir(L.atoms[0]) if not x.startswith('__')]

In [ ]:
L.atoms[0].position

In [ ]:
L.atoms[0].id

In [ ]:
L.atoms[0].velocity

In [ ]:
L.atoms[0].force

In [ ]:
L.atoms[0].type